In [40]:
#import data

# -*- coding: utf-8 -*-

#import data

import pandas as pd
import numpy as np
import torch.onnx
import os

import torch
import torch.nn as nn
# import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms



In [41]:
# 파일이 위치한 디렉토리 경로
directory_path = r'C:\Users\kyos1\Desktop\Neural Network data\Pytorch_train_data'


In [42]:
all_x_data = []
all_y_data = []

for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory_path, filename)
        # Pandas를 이용해 파일 읽기
        data = pd.read_csv(file_path, sep=",", header=0, dtype=float)
        x_data = torch.tensor(data.iloc[:, [0,2,3,4,5]].values, dtype=torch.float32)
        y_data = torch.tensor(data.iloc[:, 1:2].values, dtype=torch.float32)
        
        # 전체 데이터 리스트에 추가
        all_x_data.append(x_data)
        all_y_data.append(y_data)
        
# 리스트에 저장된 모든 데이터를 하나의 텐서로 결합
all_x_data = torch.cat(all_x_data, dim=0)
all_y_data = torch.cat(all_y_data, dim=0)

# 전체 데이터에 대한 평균과 분산 계산
mean_x = torch.mean(all_x_data, dim=0)
std_x = torch.std(all_x_data, dim=0)

Max_Vy = max(abs(all_y_data))

In [43]:
x_data = (all_x_data - mean_x) / std_x  # mean과 std는 훈련 데이터셋에서 계산된 값 사용
y_data = all_y_data/float(Max_Vy)

# 배치사이즈 설정
batch_size = 512

In [44]:
class My_dataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):  #데이터 셋 선처리 해주는 부분
        self.x_data = x_data.unsqueeze(1)
        self.y_data = y_data
        self.transform = transform
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        x = self.x_data[index]  # (1, input_size) 형식
        y = self.y_data[index]  # (1) 형식

        if self.transform:
            x, y = self.transform((x, y))
        return x, y

    def __len__(self):
        return self.len

class ToTensor:
    def __call__(self, sample):
        x, y = sample
        x = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        return x, y


# 데이터 로드
trans = transforms.Compose([ToTensor()])

train_data = My_dataset(x_data, y_data, transform=trans)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=True)

In [45]:
# LSTM 모델 정의
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # 마지막 시점의 출력만 사용
        out = self.tanh(out)  # tanh 활성화 함수 적용
        return out
    
    
# 모델 초기화 및 손실 함수, 옵티마이저 설정
input_size = 5
hidden_size = 64
num_layers = 2
output_size = 1

model = LSTMNet(input_size, hidden_size, num_layers, output_size)

In [46]:
Loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.99)

In [47]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

51521


In [48]:
losses = []

# 훈련 루프
num_epochs = 600
num_data = len(x_data)


min_loss = 50
training_epoch = 0

try:
    for epoch in range(num_epochs):
        losses = []
        epoch_losses = []
        average_losses = []

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            inputs = inputs  # LSTM 입력 형태에 맞게 변경 (batch_size, seq_len, input_size)
            outputs = model(inputs)
            targets = targets.view(-1, 1)
            
            loss = Loss(outputs, targets)
            loss.backward()
            
            optimizer.step()
            epoch_losses.append(loss.item())
    
        average_losses = np.mean(epoch_losses)
        losses.append(average_losses)
    
        if average_losses < min_loss:
            min_loss = average_losses
            model.eval()  # 모델 평가 모드로 전환
            x_dummy = torch.randn(1, 1, input_size)
            traced_model = torch.jit.trace(model, x_dummy)
            torch.jit.save(traced_model, r'C:\Users\kyos1\Desktop\Neural Network data\0531_model_1.pt')

        if epoch in [100,200,300,400]:
            model.eval()  # 모델을 평가 모드로 설정
            x_dummy = torch.randn(1, 1, input_size)
            traced_model = torch.jit.trace(model, x_dummy)
            torch.jit.save(traced_model, rf'C:\Users\kyos1\Desktop\Neural Network data\0531_model_{epoch}.pt')

        # Scheduler 업데이트
        scheduler.step()
        training_epoch = epoch
        # 진행 상황 출력 (선택적)
        print(f"Epoch {epoch}, average_losses: {average_losses}")        

except KeyboardInterrupt:
    print("Training interrupted. Saving the model.")
    model.eval()  # 모델을 평가 모드로 설정
    x_dummy = torch.randn(1, 1, input_size)
    traced_model = torch.jit.trace(model, x_dummy)
    # 트레이싱된 모델 저장
    torch.jit.save(traced_model, r'C:\Users\kyos1\Desktop\Neural Network data\0531_model_2.pt')

# 모델 저장
model.eval()  # 모델을 평가 모드로 설정
x_dummy = torch.randn(1, 1, input_size)
traced_model = torch.jit.trace(model, x_dummy)

# 트레이싱된 모델 저장
torch.jit.save(traced_model, r'C:\Users\kyos1\Desktop\Neural Network data\0531_model_3.pt')
print("Traced model saved.")

Epoch 0, average_losses: 0.025432577548891236
Epoch 1, average_losses: 0.02108662089916113
Epoch 2, average_losses: 0.019067802205478622
Epoch 3, average_losses: 0.017858773277283652
Epoch 4, average_losses: 0.01684435142244252
Epoch 5, average_losses: 0.015876680832078355
Epoch 6, average_losses: 0.014948752327912205
Epoch 7, average_losses: 0.014062456646055545
Epoch 8, average_losses: 0.013206050445763328
Epoch 9, average_losses: 0.012359973805256953
Epoch 10, average_losses: 0.011504934452016145
Epoch 11, average_losses: 0.010626893207202401
Epoch 12, average_losses: 0.009719225106102455
Epoch 13, average_losses: 0.008783455147286592
Epoch 14, average_losses: 0.007829532290002736
Epoch 15, average_losses: 0.006875639021361493
Epoch 16, average_losses: 0.005946992275020643
Epoch 17, average_losses: 0.005073085563880049
Epoch 18, average_losses: 0.0042832242928990075
Epoch 19, average_losses: 0.003601028421676261
Epoch 20, average_losses: 0.0030395064683068755
Epoch 21, average_losse